In [1]:
import scapy
from scapy.all import sniff, get_if_list
from collections import Counter
import pandas as pd
import socket
import os

In [2]:
pd.options.display.width = 100

In [3]:
def get_packets(interface = "ethernet_0", filename = "sniff_cap.csv", count = 100, print_status = False, timeout=None):
    """
    This function is designed to capture a number of ip packets to analyize with pandas.
    """
    #Checks if file exists
    if os.path.isfile(filename):
        df = pd.read_csv(filename)
        return df
    #Sniffs on all interfaces for 'count' number of ip packets
    else:    
        sniffed_packets = sniff(iface=interface, count=count, filter="ip", timeout=timeout)
        list_of_packets = []
        i = 0

        for packet in sniffed_packets:
            if packet.haslayer('IP'):
                try:
                    source_name = socket.gethostbyaddr(packet['IP'].src)
                except socket.herror:
                    source_name = None
                try:
                    destination_name = socket.gethostbyaddr(packet['IP'].dst)
                except socket.herror:
                    destination_name = None
                i += 1
                #Keeps track of how many captures and prints to screen
                if print_status == True:
                    if i % (count/10) == 0:
                        print(f"Percent complete: {(i/count) * 100} %")

                #Collects the information into a dictionary for later use in Pandas
                temp_dict = {"src_IP" : str(packet['IP'].src),
                             "src_port" : str(packet['TCP'].sport),
                             "src_name": source_name,
                             "dst_IP" : str(packet['IP'].dst),
                             "dst_port" : str(packet['TCP'].dport),
                             "dst_name": destination_name,
                             "payload": str(packet['IP'].payload)}
                list_of_packets.append(temp_dict)
                
        df = pd.DataFrame(list_of_packets)
        df.to_csv(filename, index=False)

        return df

In [4]:
list_of_ifaces0 = get_if_list()

In [5]:
usable_ifaces = []

In [6]:
for iface in list_of_ifaces0:
    try:
        sniff(iface=iface, count=1, timeout=1)
        usable_ifaces.append(iface)
    except OSError:
        pass

In [7]:
#print(usable_ifaces)

In [8]:
i = 0
for item in usable_ifaces:
    sniffer = get_packets(interface=item, filename = f"sniff_{i}.csv", timeout=60)
    #print(sniffer['src_port'].value_counts().head(3))
    #print(sniffer['dst_port'].value_counts().head(3))
    print("-----------------------------------------------------------")
    i+= 1

-----------------------------------------------------------


In [9]:
#df.shape